# Hyperparameter Optimization
Created by Andy Smet, Kristof Egghe


We load the 5 random train test splits and for each train test split we optimize the hyper parameters and choose the ones with the highest recall. Then we calculate the mean recall an ndcg and the standard deviations between the 5 train test splits and look at the optimal hyperparameters for each spli.

In [16]:
# set to the correct paths

allData=False


path="/home/andy/Desktop/AI_Project/"
# path="./drive/MyDrive/AI Project/"


interactions_csv_path = path+"datasets/TestDataPreprocessed_interactions.csv"
# goodreads_path = '/home/andy/Desktop/AI_Project/datasets/Goodreads/'
folds_path=path+"datasets/"
consgroup_path=path+"datasets/TestDataConsGroupdict.pickle"
reversegroup_path=path+"datasets/TestDataReverseGroupdict.pickle"
groupdict_path=path+"datasets/TestDataGroupdict.pickle"
userdic_path=path+"datasets/TestDataUser_dic.pickle"
books_df_g_path=path+"datasets/books_df_grouped.csv"


In [17]:
# Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

# EASER modified to work with groups
Easer normally sets the self similarity to zero to avoid items recommending themselves, but when all volumes are combined into one group we do want those groups to be similar to themselves so we set the diagonal to zeros except for the possitions of the grouped volumes


Calculating the inverse of the dense item to item matrix is very expensive memory wise. To combat this we use a sparse approximation https://arxiv.org/pdf/1904.13033.pdf#page=6

In [22]:

#Embarrassingly Shallow Autoencoder
from scipy.sparse import find
import itertools
import re
import numpy as np
class Ease():
    def __init__(self, K=10, l=1000):
        #how many predictions to make
        self.K = K
        #lambda L2 regularization term

        self.l = l
        #weight matrix B
        self.B = None
        #threshold for corrcoeff matrix in fit
        self.threshold = 0.65
        #new diag that helps for adjusted self-similarity scores
        self.newDiag = []

    def fit(self, X):
        #implementation of sparse inverse calculations
        #get ItemxItem gram matirx G
        G = (X.T @ X).toarray().astype(np.float32)
        print("multiplication klaar")
        #calculate corrcoef matrix on G
        c=np.corrcoef(G).astype(np.float32)
        c = np.nan_to_num(c)
        
        print("corr klaar")
        #set threshold for maximal size of blocks
        maxi=1000
        # threshold=0.3 -> 850 items worden gerecommend
        # maxi=1500
        #first set all items in the corrcoef matrix that are above a threshold to 1
        for row in c.T:
          l=list(reversed(np.argsort(row)))[:maxi]
          for i in l:
            if row[i]>self.threshold:
              row[i]=1
            else:
              break
        #binarize corrcoef matrix
        A=np.where(c==1, 1, 0)
        

        #get list of indices in descending order of column sums
        res=A.sum(axis=0)
        
        l=np.array(list(reversed(np.argsort(res))))
        setje=set()
        #iterate over indices and generate submatrices
        spars=scipy.sparse.lil_matrix(G.shape)
        for i in l:
          if i in setje:
              continue
          #get itemindexes where element == 0
          col=A[:,i]
          itemindexes=np.where(col==1)
          #construct submatrix
          sub=G[np.ix_(itemindexes[0],itemindexes[0])]
          diag=scipy.sparse.dia_matrix(sub.shape)
          #set submatrixs diagonal to lambda + submatrix
          diag.setdiag([self.l]*sub.shape[0])
          sub2=(sub+diag)
          #calculate inverse and set diagonal to 0 (self-similarity)
          P=np.linalg.inv(sub2)
          B=-P/P.diagonal()
         
          
          np.fill_diagonal(B,0)
          # store values in sparse matrix
          for i in itertools.product(range(B.shape[0]),range(B.shape[0])):
            i1=i[0]
            j1=i[1]
            spars[itemindexes[0][i1],itemindexes[0][j1]]=B[i1,j1]
          for j in itemindexes[0]:
              setje.add(j)
        # d = [0 if len(reverseGroupdic[key])==1 else similarityScoreDic[key] for key in similarityScoreDic]
        spars.setdiag(self.newDiag)
        self.B = spars.toarray()
    def predict(self, X, firstUserID):
        
        S = X*self.B
        users = set(X.nonzero()[0])
        
        print(firstUserID)
      
        U, I, V = [], [], []
        real_groups=list(cons_group_dic.keys())

        for user in users:
            history_of_user=randomSplit_df.loc[randomSplit_df['user_id']==user+firstUserID].history.item()
            U.extend([user] * self.K)
            

            # a=np.array([S[user][i] if i not in history_of_user or (len(reverseGroupdic[i]) > 1 and len(re.split(' Volume| Vol\.| vol\.| #', books_df_g.loc[books_df_g['group_id'] == list(cons_group_dic.keys())[i]].title.item())) > 1) else -self.l*2 for i in range(len(S[user])) ])
            a = []
            for i in range(len(S[user])):
             
              try:
                lastViewedIndex = (len(history_of_user) - 1 - history_of_user[::-1].index(i))
              except:
                lastViewedIndex = len(history_of_user)
              if i not in history_of_user or (len(reverseGroupdic[real_groups[i]]) > 1 and len(re.split(' Volume| Vol\.| vol\.| #', books_df_g.loc[books_df_g['group_id'] == real_groups[i]].title.item())) > 1):
                a.append(S[user][i]*((lastViewedIndex/len(history_of_user))**2)) 
              else:
                a.append(-self.l*2)
            # a = np.array([S[user][i]*history_of_user.count(i) if (len(reverseGroupdic[i]) > 1 and history_of_user.count(i)<len(reverseGroupdic[i])) else a[i] for i in range(len(a))])
            items = np.array(a).argsort()[-self.K:][::-1]
            I.extend(items)
            V.extend(S[user][items])

        score_matrix = scipy.sparse.csr_matrix((V, (U, I)), shape=X.shape).astype(np.float32)
        print("predict klaar")
        return score_matrix



# Hyperparameter Optimization For each split

In [32]:
from scipy.sparse import vstack
from sklearn.model_selection import KFold
import sklearn.metrics
import statistics
import pickle
import pandas as pd
with open(consgroup_path, 'rb') as f:
    cons_group_dic= pickle.load(f)
with open(groupdict_path, 'rb') as f:
    groupdic= pickle.load(f)
with open(reversegroup_path, 'rb') as f:
    reverseGroupdic= pickle.load(f)
with open(userdic_path, 'rb') as f:
    userdic= pickle.load(f)
randomSplit_df=0
books_df_g=pd.read_csv(books_df_g_path)

display(books_df_g)
def recall_easer_hyperparameter_optimization(model,K):
  global randomSplit_df

  ease = model
  recalls = []
  ndcgs = []
  y_pred_totals = []
  hyperParams=[]
  for s in range(4,6):
    if allData:
      randomSplit_df = pd.read_csv(folds_path+"AllDatafold"+str(s)+".csv")
    else:
      randomSplit_df = pd.read_csv(folds_path+"TestDatafold"+str(s)+".csv")
    display(randomSplit_df.head(20))
    display(randomSplit_df[randomSplit_df["item_id"].apply(len).lt(20)])

    randomSplit_df['user_id'] = randomSplit_df.user_id.apply(lambda x:int(userdic[int(x)]))

    randomSplit_df['history'] = randomSplit_df.history.apply(lambda x:[int(cons_group_dic[groupdic[int(y)]]) if y else y for y in x[1:-1].split(',')])
    randomSplit_df['future'] = randomSplit_df.future.apply(lambda x:[int(cons_group_dic[groupdic[int(y)]]) if y else y for y in x[1:-1].split(',')])
    shape = (randomSplit_df['user_id'].max() +1, len(cons_group_dic))

    ease.newDiag = get_newDiag(randomSplit_df,shape)
    train_x = create_sparse_matrix(randomSplit_df, column='history', shape=shape)
    y_true = create_sparse_matrix(randomSplit_df, column='future', shape=shape)
   
    
    recalls_fold = []
    ndcgs_fold=[]
    hyperParams_fold=[]
    for lambda_ in [250000,2500000]:
      for threshold in [0.9]:
        ease.threshold = threshold
        ease.l=lambda_
        ease.fit(train_x)

        interval=int(train_x.shape[0]/20)
        true_ndcg=0
        true_recall=0
        y_pred_total = scipy.sparse.csr_matrix((0,train_x.shape[1])).astype(np.float32)
        
        for j in range(20-1):
          predict_split=train_x[np.ix_(list(range(interval*j,interval*(j+1))),list(range(train_x.shape[1])))]
          true_split=y_true[np.ix_(list(range(interval*j,interval*(j+1))),list(range(train_x.shape[1])))]
          y_pred = ease.predict(predict_split, interval*j)
          y_pred_total = vstack((y_pred_total, y_pred))
          scores = scipy.sparse.lil_matrix(y_pred.shape)
          scores[y_pred.multiply(true_split).astype(bool)] = 1
          scores = scores.tocsr()
          scores = sparse_divide_nonzero(scores, scipy.sparse.csr_matrix(true_split.sum(axis=1))).sum(axis=1)
          true_recall+=scores.mean()*(interval/train_x.shape[0])
      
          ndcg = sklearn.metrics.ndcg_score(true_split.toarray(), y_pred.toarray(), k=K)
          true_ndcg+=ndcg*(interval/train_x.shape[0])

        predict_split=train_x[np.ix_(list(range(interval*19,train_x.shape[0])),list(range(train_x.shape[1])))]
        true_split=y_true[np.ix_(list(range(interval*19,train_x.shape[0])),list(range(train_x.shape[1])))]
        y_pred = ease.predict(predict_split, interval*19)
        y_pred_total = vstack((y_pred_total, y_pred))
        scores = scipy.sparse.lil_matrix(y_pred.shape)
        scores[y_pred.multiply(true_split).astype(bool)] = 1
        scores = scores.tocsr()
        scores = sparse_divide_nonzero(scores, scipy.sparse.csr_matrix(true_split.sum(axis=1))).sum(axis=1)
        true_recall+=scores.mean()*(interval/train_x.shape[0])

        
        ndcg = sklearn.metrics.ndcg_score(true_split.toarray(), y_pred.toarray(), k=K)
        true_ndcg+=ndcg*(interval/train_x.shape[0])
        print("recall @ {}: {:.4f}, ndcg: {:.4f} params lambda:{}, threshold={}".format(K,true_recall,true_ndcg ,lambda_, threshold))

        
        ndcgs_fold.append(true_ndcg)
        recalls_fold.append(true_recall)
        hyperParams_fold.append((lambda_, threshold))
    max_value = max(recalls_fold)
    max_index = recalls_fold.index(max_value)
    recalls.append(recalls_fold[max_index])
    ndcgs.append(ndcgs_fold[max_index])
    hyperParams.append(hyperParams_fold[max_index])
  
  meanRecall = statistics.mean(recalls)
  stdevRecall = statistics.stdev(recalls)

  meanNdcg = statistics.mean(ndcgs)
  stdevNdcg = statistics.stdev(ndcgs)
  print("mean recall @ {}: {:.4f}, stdev: {:.4f}".format(K, meanRecall, stdevRecall))
  print("mean ndcg @ {}: {:.4f}, stdev: {:.4f}".format(K, meanNdcg, stdevNdcg))
  print("used hyperParameters (lambda, threshold): {}".format(hyperParams))
  return meanNdcg,meanRecall, hyperParams
  

,book_id,title,authors,publisher,num_pages,publication_year,author_name,group_id
0,[1242262],""" Superman Returns "": The Visual Guide","[{'author_id': '4953553', 'role': ''}]",{''},70,2006.0,Daniel Wallace,0
1,[23491474],"""A"" come ignoranza: allenamento anti papero","[{'author_id': '5559315', 'role': ''}]",{'Panini Comics'},64,2014.0,Daw,1
2,[23491452],"""A"" come ignoranza: esplosione di esplosioni","[{'author_id': '5559315', 'role': ''}]",{'Panini Comics'},64,2014.0,Daw,2
3,[24542438],"""A"" come ignoranza: principesse crude","[{'author_id': '5559315', 'role': ''}]",{'Panini Comics'},64,2015.0,Daw,3
4,[23491481],"""A"" come ignoranza: spoiler! Sbranzo muore","[{'author_id': '5559315', 'role': ''}]",{'Panini Comics'},64,2014.0,Daw,4
...,...,...,...,...,...,...,...,...
50360,[10302849],ＧＥ～グッドエンディング～[Guddo Endingu](GE: Good Ending #1),"[{'author_id': '5592402', 'role': ''}]",{''},NaN,NaN,Kei Sasuga,50360
50361,[18869719],Ｌ・ＤＫ（１０） (講談社コミックス別冊フレンド),"[{'author_id': '1262831', 'role': ''}]",{''},NaN,NaN,Ayu Watanabe,50361
50362,[18869791],Ｌ・ＤＫ（１２）,"[{'author_id': '1262831', 'role': ''}]",{''},NaN,NaN,Ayu Watanabe,50362
50363,[23829170],ＳＡＭＵＲＡＩ ＤＥＥＰＥＲ ＫＹＯ（１）,"[{'author_id': '198197', 'role': ''}, {'author...",{''},NaN,NaN,Akimine Kamijyo,50363


In [35]:
import scipy.sparse

#Create scipy csr matrix
def create_sparse_matrix(sessions_df, column='history', shape=None):
  #flatten
  user_ids = []
  item_ids = []
  for idx, row in sessions_df.iterrows():
    items = row[column]
    user = row['user_id']
    user_ids.extend([user] * len(items))
    item_ids.extend(items)
  #create csr matrix
  values = np.ones(len(user_ids))
  print(user_ids[:20])
  print(item_ids[:20])
  matrix = scipy.sparse.csr_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
  return matrix


def get_newDiag(df,shape):
  self_similarity_n={}
  self_similarity={}
  for idx,row in df.iterrows():
    l=row['history']
    histset=set(l)
    real_group_id_list=list(cons_group_dic.keys())
    for concgroup in histset:
      # print(concgroup,histset)
      if not concgroup:
        continue
      real_group_id=real_group_id_list[concgroup]
      grouplist=reverseGroupdic[real_group_id]
      if len(grouplist) ==1:
        self_similarity[concgroup]=0
        # self_similarity_n[concgroup]=1

      else:
        self_similarity[concgroup]=1
  # for i in self_similarity:
  #   self_similarity[i]=self_similarity[i]/(self_similarity_n[i]+20)
  new_diag=[self_similarity[i] if i in self_similarity else 0 for i in list(range(shape[1]))]
  return new_diag

In [36]:
def sparse_divide_nonzero(a: scipy.sparse.csr_matrix, b: scipy.sparse.csr_matrix) -> scipy.sparse.csr_matrix:
    return a.multiply(sparse_inverse_nonzero(b))

def sparse_inverse_nonzero(a: scipy.sparse.csr_matrix) -> scipy.sparse.csr_matrix:
    inv_a = a.copy()
    inv_a.data = 1 / inv_a.data
    return inv_a


K = 10
for K in [10]:
  
  ease = Ease(K=K)
  ndcg,recall, y_pred =recall_easer_hyperparameter_optimization(ease,K)

,user_id,item_id,datetime,history,future
0,10,"[5173602, 110918, 96358, 5418388, 16992, 18619...","[Timestamp('2012-10-01 11:08:19'), Timestamp('...","[472331, 51080, 77318, 2249571, 6086315, 18215...","[22369, 160841, 2637138, 2249571, 1119044, 198..."
1,13,"[16791, 11909762, 359193, 314573, 437113, 1518...","[Timestamp('2008-07-26 08:34:14'), Timestamp('...","[15719, 437094, 160471, 359196, 1124521, 47697...","[25102, 160471, 1124521, 209966, 2186848, 1025..."
2,16,"[18465566, 20256612, 20578979, 23128353, 69797...","[Timestamp('2015-04-05 18:03:44'), Timestamp('...","[71252, 25849, 29936927, 25566235, 22605161, 7...","[20870790, 15736709, 392297, 1725532, 6839093,..."
3,17,"[392297, 13191639, 15704307, 15800522, 2248795...","[Timestamp('2013-02-11 03:48:55'), Timestamp('...","[419103, 26778329, 24796699, 29396738, 3133872...","[31338724, 28204534, 16146214, 25066765, 29396..."
4,23,"[472331, 9607987, 13542990, 9516, 13602241, 28...","[Timestamp('2016-04-01 07:14:04'), Timestamp('...","[10974311, 24188496, 9260507, 2159700, 1787523...","[21532247, 29071035, 2272713, 2159700, 2869537..."
5,26,"[18659623, 25747291, 31219059, 18373313, 18404...","[Timestamp('2015-08-13 23:45:26'), Timestamp('...","[25241697, 13227319, 28109971, 23012564, 17671...","[28109971, 17671939, 29653775, 27809600, 21519..."
6,28,"[6238080, 7874538, 15984353, 707742, 11081964,...","[Timestamp('2012-10-29 15:35:05'), Timestamp('...","[7619398, 25074849, 18359942, 29246432, 183733...","[25066765, 12029129, 7106740, 20898018, 204055..."
7,38,"[871, 866, 873, 10248518, 6408558, 9325, 52667...","[Timestamp('2012-09-14 23:18:16'), Timestamp('...","[4341085, 26422, 6065429, 44734, 8062063, 869,...","[870, 2855165, 868, 44734, 82637, 10281904]"
8,41,"[71292, 1382693, 146123, 165556, 146104, 96428...","[Timestamp('2007-06-27 10:53:13'), Timestamp('...","[87425, 96428, 146109, 146108, 6443100, 179172...","[146122, 146107, 146108, 146160, 192043]"
9,42,"[25451555, 25853351, 1521926, 23200006, 175715...","[Timestamp('2014-06-04 18:35:45'), Timestamp('...","[21532236, 28862528, 23200006, 8596384, 255461...","[23754, 28862528, 25546167, 21433253, 20898019..."


,user_id,item_id,datetime,history,future


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[5, 11, 3, 18, 23, 15, 7, 2, 18, 21, 8, 23, 21, 0, 18, 14, 22, 19, 17, 16]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]
[14, 21, 1, 18, 21, 2, 30, 24, 25, 43, 39, 39, 34, 33, 182, 126, 113, 245, 78, 252]
multiplication klaar


/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


corr klaar
0
predict klaar
369
predict klaar
738
predict klaar
1107
predict klaar
1476
predict klaar
1845
predict klaar
2214
predict klaar
2583
predict klaar
2952
predict klaar
3321
predict klaar
3690
predict klaar
4059
predict klaar
4428
predict klaar
4797
predict klaar
5166
predict klaar
5535
predict klaar
5904
predict klaar
6273
predict klaar
6642
predict klaar
7011
predict klaar
recall @ 10: 0.2824, ndcg: 0.5484 params lambda:250000, threshold=0.9
multiplication klaar


...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 1, 18, 7, 12, 22, 2, 14, 15, 10, 21, 9, 21, 19, 6, 0, 16, 4, 13, 18]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]
[5, 21, 9, 18, 15, 18, 44, 30, 29, 31, 40, 39, 43, 35, 282, 257, 91, 194, 299, 123]
multiplication klaar


/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


corr klaar
0
predict klaar
369
predict klaar
738
predict klaar
1107
predict klaar
1476
predict klaar
1845
predict klaar
2214
predict klaar
2583
predict klaar
2952
predict klaar
3321
predict klaar
3690
predict klaar
4059
predict klaar
4428
predict klaar
4797
predict klaar
5166
predict klaar
5535
predict klaar
5904
predict klaar
6273
predict klaar
6642
predict klaar
7011
predict klaar
recall @ 10: 0.2736, ndcg: 0.5309 params lambda:250000, threshold=0.9
multiplication klaar


/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/andy/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


corr klaar
0
predict klaar
369
predict klaar
738
predict klaar
1107
predict klaar
1476
predict klaar
1845
predict klaar
2214
predict klaar
2583
predict klaar
2952
predict klaar
3321
predict klaar
3690
predict klaar
4059
predict klaar
4428
predict klaar
4797
predict klaar
5166
predict klaar
5535
predict klaar
5904
predict klaar
6273
predict klaar
6642
predict klaar
7011
predict klaar
recall @ 10: 0.2756, ndcg: 0.5315 params lambda:2500000, threshold=0.9
mean recall @ 10: 0.2831, stdev: 0.0107
mean ndcg @ 10: 0.5423, stdev: 0.0152
used hyperParameters (lambda, threshold): [(2500000, 0.9), (2500000, 0.9)]
